### week 6

# Bayesian Regression

- overdetermined and underdetermined systems
- Bayesian inference: prior and posterior
- multivariate Bayesian regression
- Bayesian approach to making prediction with Olympic marathon data

In [1]:
import pods
import notebook as nb
import mlai
import numpy as np
import scipy as sp
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
basis = mlai.polynomial
data = pods.datasets.olympic_marathon_men()
x = data['X']
y = data['Y']
data_limits = [1892, 2020]
num_data = x.shape[0]
max_basis = y.shape[0]

### Review and Preview

- Last week we studied an over-fitting problem in machine learning and explore how to validate the model
- We tested hold out validation schemes, including leave one out error and $k$-fold cross validation
- This week we look at Bayesian regression

### Two Simultaneous Equations

Given two observations $(x_1,y_1)$ and $(x_2,y_2)$, a system of two simultaneous equations with two unknowns $m$ and $c$ :
\begin{align*}
  y_1 = & mx_1 + c \\
  y_2 = & mx_2 + c
\end{align*}
is leading to a solution:
\begin{align*}
  m^* = & \frac{y_2-y_1}{x_2-x_1} \\
  c^* = & y_1 - m^* x_1
\end{align*}

### Overdetermined System (Review)

How do we deal with three observations with only two unknowns?
\begin{align*}
  y_1 = & mx_1 + c \\
  y_2 = & mx_2 + c \\
  y_3 = & mx_3 + c
\end{align*}

### Overdetermined System (Review)

In ** week 3** we solved this problem through introduction of a Gaussian noise model, ie, given $\epsilon \sim \mathcal{N}(0,\sigma^2)$ :
\begin{align*}
  y_1 = & mx_1 + c + \epsilon_1 \\
  y_2 = & mx_2 + c + \epsilon_2 \\
  y_3 = & mx_3 + c + \epsilon_3
\end{align*}

- a noise model to give mismatch between model and data
- model justified by appeal to the **central limit theorem**
- maximum likelihood with Gaussian noise leading to **least squares**

### Underdetermined System

What about two unknowns but only one observation?
$$
  y_1 =  mx_1 + c
$$

(next graph) lines through a single point $(1,3)$

In [3]:
nb.display_plots('under_determined_system{samp:0>3}.svg', directory='./diagrams', samp=(0, 9))

### Underdetermined System

We can compute $m$ given $c$ :
$$
  m^* = \frac{y_1 - c}{x_1}
$$
where we may assume
$$
  c \sim \mathcal{N}(0,\alpha_1)
$$

### Prior Distribution

Bayesian inference requires a prior on the parameters.

- the prior to represent our belief *before* we see the data of the likely value of the parameters
- for linear regression, consider a Gaussian prior:
$$
  c \sim \mathcal{N}(0, \alpha_1)
$$

### Posterior Distribution

Posterior distribution is found by combining the prior with the likelihood.

- posterior distribution to represent our belief *after* we see the data of the likely value of the parameters
- the posterior being derived through **Bayes' rule**:
$$
  p(c|y) = \frac{p(y|c)p(c)}{p(y)}
$$

### Prior and Likelihood are Both Exponentiated Quadratics

We consider a Gaussian prior $c \sim \mathcal{N}(0,\alpha_1)$ :
$$
  p(c) = \frac{1}{\sqrt{2\pi\alpha_1}} \exp\left( -\frac{c^2}{2\alpha_1} \right)
$$

We saw in **week 3** that, by using a linear regression model $y_i = mx_i + c + \epsilon_i$ with $\epsilon_i \sim \mathcal{N}(0,\sigma^2)$ for a data set $(\mathbf{x},\mathbf{y})$, it has resulted in the following likelihood function:
$$
  p(\mathbf{y} | \mathbf{x},m,c,\sigma^2) = \frac{1}{\left(2\pi\sigma^2\right)^{\frac{n}{2}}} \exp\left( -\frac{\sum_{i=1}^n (y_i-mx_i-c)^2}{2\sigma^2} \right)
$$

### Stages to Derivation of the Posterior

1. multiply likelihood by prior

   - when they are both *exponentiated quadratics*, the answer is always also an *exponentiated quadratic* because $\exp(a^2)\exp(b^2) = \exp(a^2 + b^2)$

2. complete the square to get the resulting density in the form of a Gaussian

3. recognise the mean and (co)variance of the Gaussian

This is the estimate of the posterior.

### Multiplying Likelihood by Prior

The Bayes rule gives
$$
  p(c | \mathbf{x},\mathbf{y},m,\sigma^2) = \frac{p(\mathbf{y} | \mathbf{x},m,c,\sigma^2) p(c)}{p(\mathbf{y} | \mathbf{x},m,\sigma^2)}
$$
We now move to logarithm:
$$
  \log p(c | \mathbf{x},\mathbf{y},m,\sigma^2) = \log p(\mathbf{y} | \mathbf{x},m,c,\sigma^2) + \log p(c) - \underbrace{\log p(\mathbf{y} | \mathbf{x},m,\sigma^2)}_{can\ be\ ignored}
$$
The last term can be ignored because
$$
  \text{argmax}_c \log p(c | \mathbf{x},\mathbf{y},m,\sigma^2) = \text{argmax}_c \{\log p(\mathbf{y} | \mathbf{x},m,c,\sigma^2) + \log p(c)\}
$$

We now complete the square of the quadratic form to obtain
\begin{align*}
  \log p(c | \mathbf{x},\mathbf{y},m,\sigma^2)
  & = \underbrace{\log\frac{1}{\left(2\pi\sigma^2\right)^{\frac{n}{2}}}}_{constant} - \frac{\sum_{i=1}^n (y_i-mx_i-c)^2}{2\sigma^2} + \underbrace{\log\frac{1}{\sqrt{2\pi\alpha_1}}}_{constant} - \frac{c^2}{2\alpha_1} - \underbrace{\log p(\mathbf{y} | \mathbf{x},m,\sigma^2)}_{can\ be\ ignored} \\
  & = -\frac{1}{2\sigma^2} \sum_{i=1}^n (y_i-mx_i)^2 + c \sum_{i=1}^n \frac{(y_i-mx_i)}{\sigma^2} - c^2 \left( \frac{n}{2\sigma^2} + \frac{1}{2\alpha_1} \right) + const. \\
  & \buildrel\triangle\over = -\frac{1}{2\tau^2}(c-\mu)^2 + const.
\end{align*}
where $\tau^2 = \left( n\sigma^{-2} + \alpha_1^{-1} \right)^{-1}$ and $\mu = \frac{\tau^2}{\sigma^2} \sum_{i=1}^n (y_i-mx_i)$ .

### Bayes Update

Why would we like to use a Gaussian prior with a Gaussian likelihood?

(note) **conjugate prior**

'In Bayesian probability theory, if the posterior distributions $p(\theta|x)$ are in the same family as the prior probability distribution $p(\theta)$, the prior and posterior are then called conjugate distributions, and the prior is called a **conjugate prior** for the **likelihood function**.'

from Wikipedia: [Conjugate prior](https://en.wikipedia.org/wiki/Conjugate_prior)

(next graph) posterior distribution is created from likelihood multiplied by prior

In [4]:
nb.display_plots('dem_gaussian{stage:0>2}.svg', './diagrams', stage=(1,3))

### The Joint Density

We really want to know the *joint* posterior density over the parameters $c$ *and* $m$.

We could now integrate out over $m$, but it is easier to consider the multivariate case.

### Bivariate Model with Height and Weight

![](./diagrams/height_weight_gaussian.svg)

###  Independence Assumption

We may assume that height and weight are independent: $p(h,w) = p(h)p(w)$

Wikipedia: [Statistical assumption](https://en.wikipedia.org/wiki/Statistical_assumption)

(next graph) assuming independence of height and weight

In [5]:
nb.display_plots('independent_height_weight{fig:0>3}.png', './diagrams', fig=(0,79))

### Independent Gaussians

\begin{align*}
  p(w, h)
  = & p(w)p(h) \\
  = & \frac{1}{\sqrt{2\pi\sigma_1^2}} \exp\left( -\frac{(w-\mu_1)^2}{2\sigma_1^2} \right) \cdot \frac{1}{\sqrt{2\pi\sigma_2^2}} \exp\left( -\frac{(w-\mu_2)^2}{2\sigma_2^2} \right) \\
  = & \frac{1}{\sqrt{2\pi\sigma_1^2}\sqrt{2\pi\sigma_2^2}} \exp\left( -\frac{1}{2} \left( \frac{(w-\mu_1)^2}{\sigma_1^2} + \frac{(h-\mu_2)^2}{\sigma_2^2} \right) \right) \\
  = & \frac{1}{\sqrt{2\pi\sigma_1^2 2\pi\sigma_2^2}} \exp\left( -\frac{1}{2} \left( \begin{bmatrix} w \\ h \end{bmatrix} - \begin{bmatrix} \mu_1 \\ \mu_2 \end{bmatrix} \right)^\top \begin{bmatrix} \sigma_1^2 & 0 \\ 0 & \sigma_2^2 \end{bmatrix}^{-1}\left( \begin{bmatrix} w \\ h \end{bmatrix} - \begin{bmatrix} \mu_1 \\ \mu_2 \end{bmatrix} \right) \right)
\end{align*}

Finally we come to
$$
  p(\mathbf{y}) = \frac{1}{\left| 2\pi\mathbf{D} \right|^{\frac{1}{2}}} \exp\left( -\frac{1}{2}(\mathbf{y} - \boldsymbol{\mu})^\top \mathbf{D}^{-1} (\mathbf{y} - \boldsymbol{\mu}) \right)
$$
by defining
$$
  \mathbf{y} = \begin{bmatrix} w \\ h \end{bmatrix} \qquad \boldsymbol{\mu} = \begin{bmatrix} \mu_1 \\ \mu_2 \end{bmatrix} \qquad \mathbf{D} = \begin{bmatrix} \sigma_1^2 & 0 \\ 0 & \sigma_2^2 \end{bmatrix}
$$

### Correlated Variables

In reality height and weight are not independent.

- body mass index (BMI) $\sim \frac{w}{h^2}$

(next graph) height and weight are correlated

In [6]:
nb.display_plots('correlated_height_weight{fig:0>3}.png', './diagrams', fig=(0,79))

### Correlated Gaussian

The form is now correlated by rotating the data space using matrix $\mathbf{R}$ :

\begin{align*}
  p(\mathbf{y})
  = & \frac{1}{\left| 2\pi\mathbf{D} \right|^{\frac{1}{2}}} \exp\left( -\frac{1}{2}(\mathbf{R}^\top\mathbf{y} - \mathbf{R}^\top\boldsymbol{\mu})^\top \mathbf{D}^{-1} (\mathbf{R}^\top\mathbf{y} - \mathbf{R}^\top\boldsymbol{\mu}) \right) \\
  = & \frac{1}{\left| 2\pi\mathbf{D} \right|^{\frac{1}{2}}} \exp\left( -\frac{1}{2}(\mathbf{y} - \boldsymbol{\mu})^\top \mathbf{R}\mathbf{D}^{-1}\mathbf{R}^\top (\mathbf{y} - \boldsymbol{\mu}) \right) \\
  = & \frac{1}{\left| 2\pi\mathbf{C} \right|^{\frac{1}{2}}} \exp\left( -\frac{1}{2}(\mathbf{y} - \boldsymbol{\mu})^\top \mathbf{C}^{-1} (\mathbf{y} - \boldsymbol{\mu}) \right)
\end{align*}
where $\mathbf{C}^{-1} = \mathbf{R} \mathbf{D}^{-1} \mathbf{R}^\top$, which gives a covariance matrix $\mathbf{C} = \mathbf{R} \mathbf{D} \mathbf{R}^\top$.


### Bayesian Approach: the Model

For a set of $n$ data points, $(\mathbf{x},\mathbf{y}) = \{(x_1,y_1), \ldots, (x_n,y_n)\}$, we start with a regression model with basis function $\boldsymbol{\Phi}$ :
$$
  \mathbf{y} = f(\mathbf{x}) + \boldsymbol{\epsilon} = \boldsymbol{\Phi}\mathbf{w} + \boldsymbol{\epsilon}
$$
where
$$
  \mathbf{y} = \left(\begin{array}{c} y_1 \\ \vdots \\ y_n \end{array}\right)
  \qquad
  \boldsymbol{\Phi} = \left(\begin{array}{c}
    \boldsymbol{\phi}_1^\top \\
    \vdots \\
    \boldsymbol{\phi}_n^\top
  \end{array}\right) = \left(\begin{array}{ccc}
    \phi_0(x_1) & \dots & \phi_m(x_1) \\
    \vdots      &       & \vdots \\
    \phi_0(x_n) & \dots & \phi_m(x_n)
  \end{array}\right)
  \qquad
  \mathbf{w} = \left(\begin{array}{c} w_0 \\ \vdots \\ w_m \end{array}\right)
  \qquad
  \boldsymbol{\epsilon} = \left(\begin{array}{c} \epsilon_1 \\ \vdots \\ \epsilon_n \end{array}\right)
$$
(reveiw) **week 4**

### Bayesian Approach:  Sampling the Prior

For a regression model, $\mathbf{y} = \boldsymbol{\Phi}\mathbf{w} + \boldsymbol{\epsilon}$ :

- assume Gaussian noise (**week 3**) for $\boldsymbol{\epsilon}$:
$$\boldsymbol{\epsilon} \sim \mathcal{N}(\mathbf{0}, \sigma^2\mathbf{I})$$

- sample the Gaussian prior for $\mathbf{w}$:
$$\mathbf{w} \sim \mathcal{N}(\mathbf{0}, \alpha\mathbf{I})$$

### Bayesian Approach: Sampling the Posterior

Recall the Bayes rule,
$$
  p(\mathbf{w} | \boldsymbol{\Phi},\mathbf{y},\sigma^2) = \frac{p(\mathbf{y} | \boldsymbol{\Phi},\mathbf{w},\sigma^2) p(\mathbf{w})}{p(\mathbf{y} | \boldsymbol{\Phi},\sigma^2)}
$$
and we are now given the prior,
$$
  p(\mathbf{w}) = \mathcal{N}(\mathbf{0}, \alpha\mathbf{I})
$$
After some effort, it eventually leads to the posteror,
$$
  p(\mathbf{w} | \boldsymbol{\Phi},\mathbf{y},\sigma^2) = \mathcal{N}(\boldsymbol{\mu}_w,\mathbf{C}_w)
$$
where
\begin{align*}
  \mathbf{C}_w = & \left( \sigma^{-2} \boldsymbol{\Phi}^\top \boldsymbol{\Phi} + \alpha^{-1} \mathbf{I} \right)^{-1} \\
  \boldsymbol{\mu}_w = & \mathbf{C}_w \sigma^{-2} \boldsymbol{\Phi}^\top \mathbf{y}
\end{align*}

### Regularised Mean

Bayesian solution of the mean:
$$
  \boldsymbol{\mu}_w = \left( \sigma^{-2} \boldsymbol{\Phi}^\top \boldsymbol{\Phi} + \alpha^{-1}\mathbf{I} \right)^{-1} \sigma^{-2} \boldsymbol{\Phi}^\top \mathbf{y}
$$

(cf) In ** week 3** we derived $\mathbf{w}^* = (\boldsymbol{\Phi}^\top \boldsymbol{\Phi})^{-1} \boldsymbol{\Phi}^\top \mathbf{y}$ .

- two are equivalent when $\alpha \rightarrow \infty$, ie, $\boldsymbol{\mu}_w$ is equivalent to  $\mathbf{w}^*$ when the variance is infinite
- in other cases $\alpha \mathbf{I}$ regularises the system, ie, keeps parameters smaller

### Marginal Likelihood

The marginal likelihood is the probability $p(\mathbf{y} | \boldsymbol{\Phi},\sigma^2)$ where $\mathbf{w}$ has been integrated out:
$$
  p(\mathbf{y} | \boldsymbol{\Phi},\sigma^2) = \int p(\mathbf{y} | \boldsymbol{\Phi},\mathbf{w},\sigma^2) p(\mathbf{w}) d\mathbf{w}
$$

(note) by denoting $\mathbf{K} = \alpha \boldsymbol{\Phi} \boldsymbol{\Phi}^\top + \sigma^2 \mathbf{I}$, further calculation leads to:
$$
  p(\mathbf{y} | \boldsymbol{\Phi},\sigma^2) = \frac{1}{(2\pi)^\frac{n}{2} |\mathbf{K}|^\frac{1}{2}} \exp\left( -\frac{1}{2} \mathbf{y}^\top \mathbf{K}^{-1} \mathbf{y} \right)
$$
which is a zero mean $n$-dimensional Gaussian with covariance $\mathbf{K}$ .

Wikipedia: [Marginal Likelihood](https://en.wikipedia.org/wiki/Marginal_likelihood)

### Making Prediction: Expected Output

Given the posterior for the parameters, how can we compute the expected output at a given location?

- output of the model at location $x_i$ is given by $f(x_i) = \boldsymbol{\phi}_i^\top \mathbf{w}$
- we want the expected output under the posterior density, $p(\mathbf{w} | \boldsymbol{\Phi},\mathbf{y},\sigma^2)$

we now calculate the expected output:
$$
  \mathbb{E}\left[ f(x_i) \right] = \int f(x_i) p(\mathbf{w} | \boldsymbol{\Phi},\mathbf{y},\sigma^2) d\mathbf{w} = \boldsymbol{\phi}_i^\top \underbrace{\int \mathbf{w} \cdot p(\mathbf{w} | \boldsymbol{\Phi},\mathbf{y},\sigma^2) d\mathbf{w}}_{\boldsymbol{\mu}_w} = \boldsymbol{\phi}_i^\top \boldsymbol{\mu}_w
$$

### Making Prediction: Variance of the Expected Output

Variance of the model at location $x_i$ is given by
\begin{align*}
  var\left[ f(x_i) \right]
  & = \mathbb{E}\left[ f(x_i)^2 \right] - \left( \mathbb{E}\left[ f(x_i) \right] \right)^2 \\
  & = \boldsymbol{\phi}_i^\top \mathbb{E}\left[ \mathbf{w} \mathbf{w}^\top \right] \boldsymbol{\phi}_i - \boldsymbol{\phi}_i^\top \mathbb{E}\left[ \mathbf{w} \right] \mathbb{E}\left[ \mathbf{w} \right]^\top \boldsymbol{\phi}_i \\
  & = \boldsymbol{\phi}_i^\top \underbrace{\left( \mathbb{E}\left[ \mathbf{w} \mathbf{w}^\top \right] - \mathbb{E}\left[ \mathbf{w} \right] \mathbb{E}\left[ \mathbf{w} \right]^\top \right)}_{\mathbf{C}_w} \boldsymbol{\phi}_i \\
  & = \boldsymbol{\phi}_i^\top \mathbf{C}_w \boldsymbol{\phi}_i
\end{align*}
where all these expectations are calculated under the posterior density, $p(\mathbf{w} | \boldsymbol{\Phi},\mathbf{y},\sigma^2)$ .

### Bayesian Estimates: Olympic Marathon Data

Here we apply a Bayesian technique on Olympic data using a regression model, $\mathbf{y} = \boldsymbol{\Phi}\mathbf{w} + \boldsymbol{\epsilon}$, with
\begin{align*}
  \boldsymbol{\epsilon} & \sim \mathcal{N}(\mathbf{0}, \sigma^2\mathbf{I}) \\
  \mathbf{w} & \sim \mathcal{N}(\mathbf{0}, \alpha\mathbf{I})
\end{align*}
where $\sigma^2 = 0.01$ and $\alpha=1$ .

(next graph) finding Bayesian polynomial fits with the number of basis between 1 and 27;
solid line shows the expected output, $\boldsymbol{\phi}_i^\top \boldsymbol{\mu}_w$, while dashed lines indicate the width for one standard diviation on the both side

In [7]:
nb.display_plots('olympic_BLM_polynomial_num_basis{num_basis:0>3}.svg', directory='./diagrams', num_basis=(1, max_basis))

### Bayesian Estimates: Hold Out Validation

(next graph) suppose we know the winning time (red circles) up to year 1984, are we able to predict those (green) for the rest of years?

In [8]:
nb.display_plots('olympic_val_BLM_polynomial_num_basis{num_basis:0>3}.svg', directory='./diagrams', num_basis=(1, max_basis))

### Bayesian Estimates: 5-fold Cross Validation

(next graph) 5-fold cross validation: validation data is in green

In [9]:
nb.display_plots('olympic_5cv{part:0>2}_BLM_polynomial_num_basis{num_basis:0>3}.svg', directory='./diagrams', part=(0, 5), num_basis=(1, max_basis))